In [6]:
import spacy
from rasa_nlu.training_data import load_data
from rasa_nlu import config
from rasa_nlu.model import Trainer,Interpreter

from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [13]:
intent_data="""
## intent: introduction
-how are you?
-howdy?
-h r u?
-hi
-hey
-hello

## intent: ask_time
-whats the time now
-can you tell me the time
-time now please?
-tell me the tim now

## intent: ask_weather
-whats the temperature in Bangalore now
-tell me the temperature in Bangalore now
-how is the weather condition in Bangalore
-can you tell me about the Bangalore weather condition

## intent: ask_hashtags
-whats are the trending hastags today
-today trending hastags please
-tell me what is trending today
-list top 10 trending hashtags
"""
%store intent_data > data.md

Writing 'intent_data' (str) to file 'data.md'.


In [9]:
spacy.load('en')

In [10]:
setting="""
pipeline: spacy_sklearn
"""

%store setting > setting.yml

Writing 'setting' (str) to file 'setting.yml'.


In [14]:
trainer=Trainer(config.load('setting.yml'))
training_data=load_data('data.md')
trainer.train(training_data)

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Administrator\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Administrator\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Administrator\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Administrator\Anaconda3\envs\tensorflow_env\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarn

In [15]:
model_directory=trainer.persist('models/',
                               fixed_model_name='current')
interpreter=Interpreter.load('models/default/current')

In [16]:
interpreter.parse('h r u')

{'intent': {'name': 'introduction', 'confidence': 0.5791276384436458},
 'entities': [],
 'intent_ranking': [{'name': 'introduction', 'confidence': 0.5791276384436458},
  {'name': 'ask_hashtags', 'confidence': 0.16865587224158804},
  {'name': 'ask_time', 'confidence': 0.13447848759977216},
  {'name': 'ask_weather', 'confidence': 0.11773800171499427}],
 'text': 'h r u'}

#### for the reply

In [17]:
domain_text="""
intents:
-introduction
-ask_time
-ask_weather
-ask_hashtags

actions:
-utter_introduce
-utter_time
-utter_weather
-utter_hashtags

templates:
  utter_introduce:
  -Welcome
  -Hi, nice to meet you
  -hello, hope you are doing good
  utter_time:
  -Your time is really bad
  -Your time is really good
  utter_weather:
  -Its cloudy
  -its sunny
  -Why do you care
  utter_hashtags:
  -#IndvsAus,#WelcomeModi,#WelcomeAbhinandan
  -#Womansday
"""
%store domain_text > domain.yml

Writing 'domain_text' (str) to file 'domain.yml'.


In [26]:
stories="""
## introduce
* introduction
  -utter_introduce

## Ask Time
* ask_time
  -utter_time

## Ask Weather
* ask_weather
  -utter_weather

## Ask hashtags
* ask_hastags
  -utter_hastags
"""
%store stories > stories.md

Writing 'stories' (str) to file 'stories.md'.


In [ ]:
Under

In [ ]:
domain_file='domain.yml'
model_path='model/dialogue'
training_data_file='stories.md'
agent=Agent(domain_file,
            policies=[MemoizationPolicy(max_history=3),KerasPolicy()])
training_data=agent.load_data(training_data_file)
agent.train(training_data)
agent.persist(model_path)

### or

In [ ]:
from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy

domain_file="domain.yml"
model_path="models/dialogue"
training_data_file="stories.md"    
agent = Agent(
    domain_file,
    policies=[MemoizationPolicy(max_history=3), KerasPolicy()]
    )
training_data = agent.load_data(training_data_file)
agent.train(
    training_data,
    epochs=400,
    batch_size=100,
    validation_split=0.2
    )
agent.persist(model_path)

### Under Bot.py

In [ ]:
from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.interpreter import RasaNLUInterpreter, NaturalLanguageInterpreter
import rasa_core

interpreter = RasaNLUInterpreter("models/default/current")
agent = Agent.load("models/dialogue", interpreter=interpreter)

def run(dbug=False):
    if dbug:
        init_debug_logging()
    interpreter = NaturalLanguageInterpreter.create("models/default/current")
    from rasa_core.utils import EndpointConfig
    action_endpoint = EndpointConfig(url="http://localhost:5056/webhook")
    agent = Agent.load("models/dialogue", interpreter=interpreter,action_endpoint=action_endpoint)
    rasa_core.run.serve_application(agent,channel='cmdline')
run()